In [1]:
import numpy as np

In [35]:
class SOM:
    def __init__(self,input_,num_of_nodes,eta,initial_wts,threshold):
        self.input_ = input_
        self.num_of_nodes = num_of_nodes
        self.eta = eta
        self.current_wts = initial_wts
        self.t2 = 1000
        self.threshold = threshold
    
    def euclidean_distance_1d(self,x,y):
        return abs(x - y)
    
    def get_winning_neuron(self,x,W):
        winner = min([(self.euclidean_distance_1d(x,w), index) for index,w in enumerate(W)])[1]
        return winner
    
    def d_ij(self,winner):
        distance = [self.euclidean_distance_1d(self.current_wts[winner],\
                                               self.current_wts[i])\
                    for i in range(len(self.current_wts))]
        return distance
    
    def gaussian(self,sigma,distance):
        h = [np.exp(-(d**2)/(2*sigma**2)) for d in distance]
        return h
    
    def compute_width(self,initial_sigma,n,t1):
        return initial_sigma*np.exp(-n/t1)
    
    def weight_adaptation(self,current_wt,eta,h,x):
        new_wts = [(w + (eta*h*(x-w))) for w in current_wt]
        return new_wts
    
    def exponential_decay_update(self,initial_eta,n,t2):
        return initial_eta*np.exp(-n/t2)
    
    def compute_t1(self,t2):
        sigma=2
        return t2/np.log(sigma)
    
    def stopping_criteria(self,w_old,w_new):
        result = 0
        for i,w in enumerate(w_old):
            result += abs(w - w_new[i])
        
        return True if (result < self.threshold) else False
    
    def train(self):
        sigma = 2
        t1 = self.compute_t1(self.t2)
        n = 1
        for x in self.input_:
            while(True):
                winning_neuron_idx = self.get_winning_neuron(x,self.current_wts)
                lateral_dist = self.d_ij(winning_neuron_idx)
                h = self.gaussian(sigma,lateral_dist)
                updates_wts = self.weight_adaptation(self.current_wts,self.eta,h[winning_neuron_idx],x)

                if not self.stopping_criteria(self.current_wts,updates_wts):
                    self.current_wts =  np.array(updates_wts)
                    self.eta = self.exponential_decay_update(self.eta,n,self.t2)
                    n += 1 
                else:
                    break
        print "Final adjusted weights :",self.current_wts

In [36]:
initial_wts = np.array([[0.15,0.45],
                        [0.3,0.9]])
inputs = [0.1,0.2,0.4,0.5]
som = SOM(inputs,2,0.1,initial_wts[0],0.01)
som.train()

Final adjusted weights : [ 0.14334454  0.4034118 ]
